In [1]:
import pandas as pd
import numpy as np

In [2]:
trypsin = pd.read_csv('../data/proteomics/isoquant/trypsin_peptide.tsv', sep = '\t')
chymotrypsin = pd.read_csv('../data/proteomics/isoquant/chymotrypsin_peptide.tsv', sep = '\t')
aspnlysc = pd.read_csv('../data/proteomics/isoquant/aspnlysc_peptide.tsv', sep = '\t')

In [3]:
trypsin['all_mapped_proteins'] = trypsin.apply(
    lambda row: row['Protein'] if pd.isna(row['Mapped Proteins']) or row['Mapped Proteins'] == '' else (row['Protein'] + ', ' + row['Mapped Proteins']),
    axis=1
)
trypsin['all_mapped_proteins'] = trypsin['all_mapped_proteins'].str.split(',')

chymotrypsin['all_mapped_proteins'] = chymotrypsin.apply(
    lambda row: row['Protein'] if pd.isna(row['Mapped Proteins']) or row['Mapped Proteins'] == '' else (row['Protein'] + ', ' + row['Mapped Proteins']),
    axis=1
)
chymotrypsin['all_mapped_proteins'] = chymotrypsin['all_mapped_proteins'].str.split(',')

aspnlysc['all_mapped_proteins'] = aspnlysc.apply(
    lambda row: row['Protein'] if pd.isna(row['Mapped Proteins']) or row['Mapped Proteins'] == '' else (row['Protein'] + ', ' + row['Mapped Proteins']),
    axis=1
)
aspnlysc['all_mapped_proteins'] = aspnlysc['all_mapped_proteins'].str.split(',')


combined = pd.concat([trypsin[['Peptide', 'all_mapped_proteins']], chymotrypsin[['Peptide', 'all_mapped_proteins']], aspnlysc[['Peptide', 'all_mapped_proteins']]])
unique_peptides = combined.drop_duplicates(subset=['Peptide'])

In [4]:
gc_count = 0
pb_count = 0
both_count = 0
    
for index,row in unique_peptides.iterrows():
    proteins = row['all_mapped_proteins']
    try:
        proteins = [i.strip() for i in proteins]
    except:
        print(row)
    
    # Check if peptide maps only to 'gc', 'pb', or both
    gc_mapped = any('gc|' in protein for protein in proteins)
    pb_mapped = any('pb|' in protein for protein in proteins)

    if gc_mapped and not pb_mapped:
        gc_count += 1
    elif pb_mapped and not gc_mapped:
        pb_count += 1
    elif gc_mapped and pb_mapped:
        both_count += 1
        
print('unqiue GC peptides: ', gc_count)
print('unqiue pb peptides: ', pb_count)
print('both: ', both_count)


unqiue GC peptides:  20723
unqiue pb peptides:  12
both:  12713


In [5]:
# How many known isoforms are validated with a unique peptide?

all_proteins = []

for index,row in unique_peptides.iterrows():
    proteins = row['all_mapped_proteins']
    proteins = [i.strip() for i in proteins]
    for i in proteins:
        all_proteins.append(i)

gencode = set([i for i in all_proteins if i.startswith('gc')])
pacbio  = set([i for i in all_proteins if i.startswith('pb')])

print('gencode: ', len(gencode))
print('pacbio: ', len(pacbio))

gencode:  15589
pacbio:  2335
